In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download rajeevsharma993/distracted-driver-dataset
! unzip distracted-driver-dataset.zip

In [ ]:
# split test folder into 3 parts
!pip install split-folders
import splitfolders
splitfolders.ratio('/content/imgs/train', output="/content/output", ratio=(0.5, 0.25, 0.25))

# import libraries and functions

In [ ]:
import os
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import pickle
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D,Input, BatchNormalization, AveragePooling2D
import random
import shutil
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from tensorflow.keras import models,optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.imagenet_utils import preprocess_input 
from keras.callbacks import EarlyStopping
import keras
import numpy as np
import tensorflow as tf


In [ ]:
path = ""
train_dir = path + "train-revamped/train/"
valid_dir = path + "validation-revamped/val/"
test_dir  = path + 'test-revamped/test/' 

In [ ]:
classes = [c for c in os.listdir(train_dir)]
classes.sort()
print(classes)

In [ ]:
classes.sort()
for c in classes:

    fig=plt.figure(figsize=(20,20))
    cl = train_dir + c
    for i,sample in enumerate(random.sample(os.listdir(train_dir + c) , 10)):
        fig.add_subplot(10, 10, i+1)
        image = plt.imread(cl + '/' + sample)

        plt.imshow(image)
        plt.xticks([])
        plt.yticks([])
        plt.title(f"{c}")

In [ ]:
classes_mapping = {   'c0' : "safe_driving",
                      'c1' : "texting-right",
                      'c2' : "talking_on_the_phone-right",
                      'c3' : "texting-left",
                      'c4' : "talking_on_the_phone-left",
                      'c5' : "operating_the_radio",
                      'c6' : "drinking",
                      'c7' : "reaching_behind",
                      'c8' : "hair-and-makeup",
                      'c9' : "talking_to_passenger"}


In [ ]:
class_names = ['Safe driving',
              'Texting - right',
              'Talking on the phone - right',
              'Texting - left',
              'Talking on the phone - left',
              'Operating the radio',
              'Drinking',
              'Reaching behind',
              'Hair and makeup',
              'Talking to passenger']

In [ ]:
for c in os.listdir(train_dir):
    shutil.move(os.path.join(train_dir,c), os.path.join(train_dir,classes_mapping[f'{c}']))

In [ ]:
for c in os.listdir(train_dir):
    os.makedirs(valid_dir + '/' + c, exist_ok=True)
    os.makedirs(test_dir + '/' + c, exist_ok=True)
    
    c_train_dir = train_dir + c
    c_len = len([sample for sample in os.listdir(c_train_dir)])
    print(c_len)
    
    for sample in random.sample(os.listdir(c_train_dir) , int(float(0.1) * c_len)):
        shutil.move(c_train_dir + '/' + sample, valid_dir + c)
        
    for sample in random.sample(os.listdir(c_train_dir) , int(float(0.1) * c_len)):
        shutil.move(c_train_dir + '/' + sample, test_dir + c)

# import data into directories

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

train_data = image_dataset_from_directory('/content/output/train',
                                       image_size=(128,128),
                                       batch_size=100,
                                       seed=123)

test_data = image_dataset_from_directory('/content/output/test',
                                       image_size=(128,128),
                                       batch_size=100,
                                       seed=123)

val_data = image_dataset_from_directory('/content/output/val',
                                       image_size=(128,128),
                                       batch_size=100,
                                       seed=123)


'''testing_data = image_dataset_from_directory('/content/imgs/',
                                       image_size=(128,128),
                                       batch_size=100,
                                       seed=123)'''

In [ ]:
train_data.class_names

In [ ]:
# assign classes to the dataset
train_data.class_names = class_names
test_data.class_names = class_names
val_data.class_names = class_names
#testing_data.class_names = class_names

In [ ]:
train_data.class_names

In [ ]:
test_data.class_names

In [ ]:
val_data.class_names

# First CNN model

In [ ]:
model=Sequential()

model.add(Conv2D(16,(3,3), activation='relu', input_shape=(128,128,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(16,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(16,activation='relu'))
model.add(Dense(10,activation='softmax'))


model.summary()

In [ ]:
# compile the model
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=["accuracy"])

In [ ]:

plot_model(model)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)
# train the model
history = model.fit(x = train_data,
          epochs=10,
          validation_data = val_data
          #,callbacks=[es]
          )

In [ ]:
plt.plot(history.history['accuracy'],label='accuracy')
plt.plot(history.history['val_accuracy'],label='validation accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'],label='loss')
plt.plot(history.history['val_loss'],label='validtion loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend()
plt.show()

In [ ]:
# saving the model 1
# pickle.dump(model, open('custom_model.pkl','wb'))

In [ ]:
# loading saved model 1
# pickled_model = pickle.load(open('/content/custom_model.pkl','rb'))

In [ ]:
# testing model 1 against the test_data
model.evaluate(test_data)

# model 2


In [ ]:
# building the layers for second model
cnn_model = Sequential()
cnn_model.add(Conv2D(32,(3,3),activation = 'relu',input_shape = (128,128,3)))
cnn_model.add(Conv2D(64,(3,3),activation = 'relu',padding = 'same'))
cnn_model.add(MaxPooling2D(2,2))
cnn_model.add(Conv2D(128,(3,3),activation = 'relu',padding = 'same'))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling2D((2,2)))
cnn_model.add(Conv2D(128,(3,3),activation = 'relu',padding='same'))
cnn_model.add(Conv2D(126,(3,3),activation = 'relu',padding='same'))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling2D((2,2)))
cnn_model.add(Conv2D(256,(3,3),activation='relu'))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling2D((2,2)))
cnn_model.add(Conv2D(256,(3,3),activation='relu'))
cnn_model.add(BatchNormalization())
cnn_model.add(Flatten())
cnn_model.add(Dense(512,activation = 'relu'))
cnn_model.add(BatchNormalization())
cnn_model.add(Dense(256,activation = 'relu'))
cnn_model.add(BatchNormalization())
cnn_model.add(Dense(256,activation = 'relu'))
cnn_model.add(BatchNormalization())
cnn_model.add(Dense(128,activation = 'relu'))
cnn_model.add(BatchNormalization())
cnn_model.add(Dense(10,activation = 'softmax',name = 'output'))
cnn_model.summary()

In [ ]:
 # compile the model 2
 cnn_model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=["accuracy"])

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)
# training the model 2
history = cnn_model.fit(x = train_data,
          epochs=10,
          validation_data = val_data,
          callbacks=[es])

In [ ]:
plt.plot(history.history['accuracy'],label='accuracy')
plt.plot(history.history['val_accuracy'],label='calidation accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'],label='loss')
plt.plot(history.history['val_loss'],label='validation loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend()
plt.show()

In [ ]:
cnn_model.evaluate(test_data)

In [ ]:
# evaluating model 2 against test data
cnn_model.evaluate(test_data)


# Third Model

In [ ]:
# add layers to 3rd model
cnn_model2 = Sequential()
cnn_model2.add(Conv2D(16,(3,3),activation = 'relu',input_shape = (128,128,3)))
cnn_model2.add(Conv2D(32,(3,3),activation = 'relu',padding = 'same'))
cnn_model2.add(AveragePooling2D())
cnn_model2.add(Conv2D(64,(3,3),activation = 'relu',padding='same'))
cnn_model2.add(Conv2D(64,(3,3),activation = 'relu',padding='same'))
cnn_model2.add(AveragePooling2D())
cnn_model2.add(Conv2D(128,(3,3),activation='relu'))
cnn_model2.add(AveragePooling2D())
cnn_model2.add(Conv2D(256,(3,3),activation='relu'))
cnn_model2.add(Flatten())
cnn_model2.add(Dense(256,activation = 'relu'))
cnn_model2.add(Dropout(0.1))
cnn_model2.add(Dense(128,activation = 'relu'))
cnn_model2.add(Dropout(0.2))
cnn_model2.add(Dense(64,activation = 'relu'))
cnn_model2.add(Dropout(0.1))
cnn_model2.add(Dense(32,activation = 'relu' ))
cnn_model2.add(Dropout(0.2))

cnn_model2.add(Dense(10,activation = 'softmax'))
cnn_model2.summary()

In [ ]:
plot_model(cnn_model2)

In [ ]:
# compile model 3
cnn_model2.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=["accuracy"])

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)

history3 = cnn_model2.fit(x = train_data,
          epochs=10,
          validation_data = val_data,
          callbacks=[es])

In [ ]:
plt.plot(history3.history['accuracy'],label='accuracy')
plt.plot(history3.history['val_accuracy'],label='valdation accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend()
plt.show()

In [ ]:
plt.plot(history3.history['loss'],label='loss')
plt.plot(history3.history['val_loss'],label='validation loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend()
plt.show()

In [ ]:
cnn_model2.evaluate(test_data)

# SAVING THE CNN MODEL IN JSON FORMAT

In [ ]:
# save model 3
cnn_model2.save('/content/model3.h5')

In [ ]:
saved_model = tf.keras.models.load_model("model3.h5")

In [ ]:
saved_model.evaluate(test_data)

# testing the model on images

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input

#load the image
my_image = load_img('/content/imgs/train/c0/img_100026.jpg', target_size=(128, 128))

#preprocess the image
my_image = img_to_array(my_image)

#make the prediction
prediction = saved_model.predict(np.array([my_image]))
res=np.argmax(prediction)
result=class_names[res]
print(result)

In [ ]:
prediction

In [ ]:
folder_path = '/content/imgs/test1'

model = cnn_model2
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# load all images into a list
images = []
for img in os.listdir(folder_path):
    img = os.path.join(folder_path, img)
    img = image.load_img(img, target_size=(128, 128))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    images.append(img)

# stack up images list to pass for prediction
images = np.vstack(images)
classes = model.class_names(images, batch_size=100)
print(classes)

In [ ]:
print(result)

In [ ]:
saved_cnn = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/saved_models/model3.h5")
saved_cnn.evaluate(test_data)

In [ ]:
saved_cnn.predict(test_data).argmax()


In [ ]:
y_true = []
y_pred = []
for x,y in test_data:
  y= tf.argmax(y,axis=1)
  y_true.append(y)
  y_pred.append(tf.argmax(saved_cnn.predict(x),axis = 1))
  
y_pred = tf.concat(y_pred, axis=0)
y_true = tf.concat(y_true, axis=0)

In [ ]:
test_data.class_names

In [ ]:

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in test_data.take(1):
  print(labels)
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
test_data.take(1)